## Convertir texto a Dataframe    

In [1]:
import pandas as pd

# Abrir el archivo de texto y leer las líneas
with open('../Dataset/Houses-dataset/Houses Dataset/HousesInfo.txt') as texto:
    contenido = texto.readlines()

# Crear una lista para almacenar los datos
datos_fila = [linea.split() for linea in contenido]

# Crear el DataFrame a partir de la lista
df = pd.DataFrame(datos_fila, columns=['bedrooms', 'bathrooms', 'area', 'zipcode', 'price'])

# Imprimir el DataFrame
df.head()


,bedrooms,bathrooms,area,zipcode,price
0,4,4,4053,85255,869500
1,4,3,3343,36372,865200
2,3,4,3923,85266,889000
3,5,5,4022,85262,910000
4,3,4,4116,85266,971226


## Agregar columna mosaico

In [2]:
import os

# Filtrar archivos con extensión .jpg
imagenes = [archivo for archivo in os.listdir('../Dataset/Houses-dataset/Houses Dataset/') if archivo.endswith('.jpg')]

# Filtrar archivos por tipo
bathrooms = [imagen for imagen in imagenes if imagen.endswith('bathroom.jpg')]
bedrooms = [imagen for imagen in imagenes if imagen.endswith('bedroom.jpg')]
kitchens = [imagen for imagen in imagenes if imagen.endswith('kitchen.jpg')]
frontals = [imagen for imagen in imagenes if imagen.endswith('frontal.jpg')]

# Ordenar las listas
bathrooms = sorted(bathrooms, key=lambda x: int(x.split('_')[0]))
bedrooms = sorted(bedrooms, key=lambda x: int(x.split('_')[0]))
kitchens = sorted(kitchens, key=lambda x: int(x.split('_')[0]))
frontals = sorted(frontals, key=lambda x: int(x.split('_')[0]))


In [9]:
# Unir las 4 imagenes para crear una imagen mosaico
from PIL import Image

# Crear una lista para almacenar las imágenes
mosaicos = []

# Iterar sobre las listas de imágenes
for i in range(len(bathrooms)):
    # Abrir las imágenes
    img1 = Image.open(f'../Dataset/Houses-dataset/Houses Dataset/{bathrooms[i]}')
    img2 = Image.open(f'../Dataset/Houses-dataset/Houses Dataset/{bedrooms[i]}')
    img3 = Image.open(f'../Dataset/Houses-dataset/Houses Dataset/{kitchens[i]}')
    img4 = Image.open(f'../Dataset/Houses-dataset/Houses Dataset/{frontals[i]}')

    # Redimensionar las imágenes a 112 x 112
    img1 = img1.resize((112, 122))
    img2 = img2.resize((112, 122))
    img3 = img3.resize((112, 122))
    img4 = img4.resize((112, 122))

    # Crear una imagen mosaico
    # baño superior izquierda, habitación superior derecha, cocina inferior izquierda, frontal inferior derecha
    mosaico = Image.new('RGB', (224, 244))
    mosaico.paste(img1, (0, 0))
    mosaico.paste(img2, (112, 0))
    mosaico.paste(img3, (0, 122))
    mosaico.paste(img4, (112, 122))

    # Renombrar la imagen mosaico
    mosaico.save(f'../Dataset/Mosaico/Imagenes/{i}.png')

    # Agregar la imagen mosaico a la lista
    mosaicos.append(f"{i}.png")


In [12]:
# Agregar la columna de imágenes mosaico al DataFrame
df['mosaic_image'] = mosaicos

# Ordenar columnas del DataFrame
df = df[['bedrooms', 'bathrooms', 'area', 'zipcode', 'mosaic_image', 'price']]

# Imprimir el DataFrame
df.head()

,bedrooms,bathrooms,area,zipcode,mosaic_image,price
0,4,4,4053,85255,0.png,869500
1,4,3,3343,36372,1.png,865200
2,3,4,3923,85266,2.png,889000
3,5,5,4022,85262,3.png,910000
4,3,4,4116,85266,4.png,971226


## Crear dataset de entrenamiento y prueba

In [13]:
from sklearn.model_selection import train_test_split

# --> Dividir dataset
X_train, X_test, y_train, y_test = train_test_split(df[['bathrooms','bedrooms','area','zipcode', 'mosaic_image']], df['price'], test_size=0.2, random_state=42)

# --> Juntar X_train e y_train
train = pd.concat([X_train, y_train], axis=1)

# --> Juntar X_test e y_test
test = pd.concat([X_test, y_test], axis=1)

# Tamños de los datasets
print(f"Train: {train.shape}")
print(f"Test: {test.shape}")


Train: (428, 6)
Test: (107, 6)


## Ver datos

In [14]:
train.head()

,bathrooms,bedrooms,area,zipcode,mosaic_image,price
284,5,5,3816,92880,284.png,589900
347,2,2,1440,92276,347.png,106000
440,3,4,1625,93510,440.png,639000
421,3,4,2454,93510,421.png,5858000
149,4.5,4,4038,92677,149.png,1795000


## Exportar datos

In [15]:
df.to_csv('../Dataset/Mosaico/dataset.csv', index=False)

train.to_csv('../Dataset/Mosaico/train.csv', index=False)
test.to_csv('../Dataset/Mosaico/test.csv', index=False)